In [1]:
import dataloaders.audio_dataset as dataset
import models.inversion_v4 as inversion_model
from abstract_model import AbstractModel

import torch
import torch.nn as nn
from torch import optim

import os

from argparse import Namespace
from pathlib import Path

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer, seed_everything

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting

from ax.service.ax_client import AxClient

from ipywidgets import IntProgress

In [2]:
AudioDataset = dataset.AudioDataset
InversionV4 = inversion_model.InversionV4

data_paths = {}
data_paths['train'] = '/scratch/prs392/incubator/data/OpenMic/train'
data_paths['val'] = '/scratch/prs392/incubator/data/OpenMic/val'
data_paths['test'] = '/scratch/prs392/incubator/data/OpenMic/test'

In [3]:
# SCRATCH = "your/scratch/location"
SCRATCH = "/scratch/prs392"
algo = "inversion_v4"
data_path = f"{SCRATCH}/incubator/data/OpenMic/"
checkpoint_path = f"{SCRATCH}/incubator/checkpoints/openl3_openmic/{algo}/"
experiment_name = "overfitting_10_audios"

In [4]:
checkpoint_path

'/scratch/prs392/incubator/checkpoints/openl3_openmic/inversion_v4/'

In [5]:
Path(checkpoint_path).mkdir(parents=True, exist_ok=True)

In [6]:
audio_dataset = AudioDataset(root_dir=data_paths['train'], num_audios = 10, return_amp = True)

for i in range(len(audio_dataset)):
    emb, spec, j = audio_dataset[i]
    print("Embeddings shape: " + str(emb.shape))
    print("Expected Spectrogram shape: " + str(spec.shape))
    m = InversionV4()
    pred = m(emb)
    print("Predicted Spectrogram shape: " + str(pred.shape))
    if i == 0:
        break

Embeddings shape: torch.Size([6144])
Expected Spectrogram shape: torch.Size([1, 128, 199])
Predicted Spectrogram shape: torch.Size([1, 1, 128, 199])


In [7]:
d = os.path.join(checkpoint_path, experiment_name)
Path(d).mkdir(parents=True, exist_ok=True)
versions = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
versions = sorted(versions)

In [8]:
from pytorch_lightning.core.saving import load_hparams_from_yaml

list_existing_hparams = []
list_of_val_loss = []

for version in versions:
    hparam_path = os.path.join(checkpoint_path, experiment_name, version, 'hparams.yaml')
    hparams_new = load_hparams_from_yaml(hparam_path)
    
    print(hparams_new)
    print(min(_load_run(os.path.join(checkpoint_path, experiment_name, version))['val_loss'][1]))
    hparams_new.pop("train_num_audios", None)    
    hparams_new.pop("val_num_audios", None)    
    hparams_new.pop("test_num_audios", None)    
    
    list_existing_hparams.append(hparams_new)
    list_of_val_loss.append(min(_load_run(os.path.join(checkpoint_path, experiment_name, version))['val_loss'][1]))
    

In [9]:
list_existing_hparams, list_of_val_loss

([], [])

In [10]:
def train_evaluate(parameterization):
    
    AudioDataset = dataset.AudioDataset
    InversionV4 = inversion_model.InversionV4
    
    data_paths = {}
    data_paths['train'] = os.path.join(data_path, 'train')
    data_paths['val'] = os.path.join(data_path, 'val')
    data_paths['test'] = os.path.join(data_path, 'test')
    
    seed_everything(123)
    
    print(parameterization)
    
    for idx, existing_hparams in enumerate(list_existing_hparams):
        shared_items = {k: existing_hparams[k] for k in existing_hparams if k in parameterization and existing_hparams[k] == parameterization[k]}
        
        if len(existing_hparams) == len(shared_items):
            print("Val loss: " + str(list_of_val_loss[idx]))
            return {'val_loss' : (list_of_val_loss[idx], 0.0)}
        
    parameterization['train_num_audios'] = 10
    parameterization['val_num_audios'] = 10
    parameterization['test_num_audios'] = 10
    parameterization['return_amp'] = True
    parameterization['num_workers'] = 10
    
    hparams = Namespace(**parameterization)
    
    model = AbstractModel(
                hparams=hparams,
                data_paths = data_paths, 
                dataset_model = AudioDataset,
                model = InversionV4(), 
                criterion = nn.MSELoss()
            )

    logger = TensorBoardLogger(checkpoint_path, name=experiment_name)

    checkpoint_callback = ModelCheckpoint(
        filepath=None,
        save_top_k=True,
        save_last = True,
        verbose=False,
        monitor='val_loss',
        mode='min',
        prefix=''
    )

    if torch.cuda.device_count() == 0:
        print('cpu')
        trainer = Trainer(
            logger=logger,
            default_root_dir=checkpoint_path,
            checkpoint_callback = checkpoint_callback,
            max_epochs=100,
            fast_dev_run=False
        )
    else:
        trainer = Trainer(
            logger=logger,
            default_root_dir=checkpoint_path,
            checkpoint_callback = checkpoint_callback,
            gpus = -1,
            distributed_backend='dp',
            max_epochs=100,
            fast_dev_run=False
        )

    trainer.fit(model)
    trainer.test(model)
    print("Val loss: " + str(model.best_validation_loss))
    return {'val_loss' : (model.best_validation_loss, 0.0)}

In [11]:
ax_client = AxClient()
ax_client.create_experiment(
    name="choose_optimizer_scheduler",
    parameters=[
        {"name": "batch_size", "type": "choice", "values": [32, 64]},
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.1], "log_scale": True},
        {"name": "lr_type", "type": "choice", "values": ['adam', 'sgd']},
        {"name": "scheduler_epoch", "type": "choice", "values": [3, 5, 7, 9]},
        {"name": "scheduler_step_size", "type": "range", "bounds": [0.1, 1.0]},
    ],
    objective_name="val_loss",
    minimize=True,
)

[INFO 08-01 13:03:30] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 2 decimal points.
[INFO 08-01 13:03:30] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.


In WithDBSettings, db settings: None


In [ ]:
total_number_of_trials = 100

for params in list_existing_hparams:
    parameters, trial_index = ax_client.attach_trial(params)
    ax_client.complete_trial(trial_index=trial_index, raw_data=train_evaluate(parameters))
    total_number_of_trials -= 1
    
for _ in range(total_number_of_trials - len(list_existing_hparams)):        
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=train_evaluate(parameters))
    

[INFO 08-01 13:03:36] ax.service.ax_client: Generated new trial 0 with parameters {'lr': 0.0, 'scheduler_step_size': 0.41, 'batch_size': 64, 'lr_type': 'adam', 'scheduler_epoch': 7}.


{'lr': 0.0014628643759539825, 'scheduler_step_size': 0.41273206453770406, 'batch_size': 64, 'lr_type': 'adam', 'scheduler_epoch': 7}


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0,1,2,3,4]
Set SLURM handle signals.

  | Name      | Type        | Params
------------------------------------------
0 | model     | InversionV4 | 16 M  
1 | criterion | MSELoss     | 0     


Set SLURM handle signals.


[INFO 08-01 13:29:15] ax.service.ax_client: Completed trial 0 with data: {'val_loss': (0.01, 0.0)}.
[INFO 08-01 13:29:15] ax.service.ax_client: Generated new trial 1 with parameters {'lr': 0.0, 'scheduler_step_size': 0.54, 'batch_size': 32, 'lr_type': 'adam', 'scheduler_epoch': 3}.


...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0,1,2,3,4]
Set SLURM handle signals.

  | Name      | Type        | Params
------------------------------------------
0 | model     | InversionV4 | 16 M  
1 | criterion | MSELoss     | 0     


Set SLURM handle signals.


[INFO 08-02 11:05:42] ax.service.ax_client: Completed trial 37 with data: {'val_loss': (0.01, 0.0)}.
[INFO 08-02 11:05:42] ax.service.ax_client: Generated new trial 38 with parameters {'lr': 0.0, 'scheduler_step_size': 0.32, 'batch_size': 64, 'lr_type': 'sgd', 'scheduler_epoch': 7}.


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(0.0186, device='cuda:0')}
--------------------------------------------------------------------------------

Val loss: 0.01402719970792532
{'lr': 6.859256528829529e-06, 'scheduler_step_size': 0.32225175835192205, 'batch_size': 64, 'lr_type': 'sgd', 'scheduler_epoch': 7}


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0,1,2,3,4]
Set SLURM handle signals.

  | Name      | Type        | Params
------------------------------------------
0 | model     | InversionV4 | 16 M  
1 | criterion | MSELoss     | 0     


In [6]:
\args = {
    'batch_size': 64,
    'lr': 0.0004136762567284789,
    'lr_type': 'sgd',
    'scheduler_epoch': 7,
    'scheduler_step_size': 0.9465236907824874,
}

args['train_num_audios'] = 10
args['val_num_audios'] = 10
args['test_num_audios'] = 10
args['return_amp'] = True
args['num_workers'] = 10

hparams = Namespace(**args)

In [7]:
hparams.lr

0.0004136762567284789

In [ ]:
seed_everything(123)

model = AbstractModel(
            hparams=hparams,
            data_paths = data_paths, 
            dataset_model = AudioDataset,
            model = InversionV4(), 
            criterion = nn.MSELoss()
        )

logger = TensorBoardLogger(checkpoint_path, name=experiment_name)
    
checkpoint_callback = ModelCheckpoint(
    filepath=None,
    save_top_k=True,
    save_last = True,
    verbose=False,
    monitor='val_loss',
    mode='min',
    prefix=''
)

if torch.cuda.device_count() == 0:
    print('cpu')
    trainer = Trainer(
        logger=logger,
        row_log_interval=100,
        log_save_interval=1000,
        val_check_interval=0.25,
        default_root_dir=checkpoint_path,
        checkpoint_callback = checkpoint_callback,
        max_epochs=100,
        fast_dev_run=False
    )
else:
    trainer = Trainer(
        logger=logger,
        row_log_interval=100,
        log_save_interval=1000,
        val_check_interval=0.25,
        default_root_dir=checkpoint_path,
        checkpoint_callback = checkpoint_callback,
        gpus = -1,
        distributed_backend='dp',
        max_epochs=5,
        fast_dev_run=False
    )

trainer.fit(model)
trainer.test(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0,1,2,3,4]
Set SLURM handle signals.

  | Name      | Type        | Params
------------------------------------------
0 | model     | InversionV2 | 7 M   
1 | criterion | MSELoss     | 0     
